In [ ]:
pip install accelerate transformers evaluate datasets peft bitsandbytes loralib

# Kaggle competition
## LLM prompt recovery

In [ ]:
#HuggingFace Login
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: ··········
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
"""
Import the necessary libraries and instantiate the instruction model
-Loading in bit8 for memory reduction
-Flash attention could be also used : attn_implementation="flash_attention_2""""
from transformers import AutoTokenizer, AutoModelForCausalLM


import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model
#Target modules obtained from documentation, these modules are frozen and used for LoRa
config = LoraConfig(
    r=8, #attention heads
    lora_alpha=32, #alpha scaling
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 9805824 || all params: 2515978240 || trainable%: 0.3897420034920493


In [ ]:
#Installing Kaggle to obtain the dataset provided by the competition.
#This test was run on Colab initially. If run in kaggle, then the following is not needed
pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle.

In [ ]:
! cp /kaggle/kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download ibrahim2002/all-in-one-dataset-with-embedding

 99% 1.24G/1.26G [00:18<00:00, 49.0MB/s]
100% 1.26G/1.26G [00:19<00:00, 70.6MB/s]


In [ ]:
! unzip /content/all-in-one-dataset-with-embedding.zip

Archive:  /content/all-in-one-dataset-with-embedding.zip
  inflating: df_with_emb.csv         
  inflating: df_with_emb.parquet     


In [ ]:
import pandas as pd
df= pd.read_csv("/content/df_with_emb.csv")

In [ ]:
df.head()

,original_text,rewrite_prompt,rewritten_text,dataset_id,original_text_emb_0,original_text_emb_1,original_text_emb_2,original_text_emb_3,original_text_emb_4,original_text_emb_5,...,rewritten_text_emb_758,rewritten_text_emb_759,rewritten_text_emb_760,rewritten_text_emb_761,rewritten_text_emb_762,rewritten_text_emb_763,rewritten_text_emb_764,rewritten_text_emb_765,rewritten_text_emb_766,rewritten_text_emb_767
0,"Dear Randy,\n\nI hope this letter finds you we...",Rephrase this letter to infuse it with an elfi...,"Dear Randy,\n\nMay this enchanted message find...",host,-0.028678,-0.072260,-0.003482,0.050595,-0.010600,-0.023786,...,-0.023307,0.011079,-0.026521,-0.035838,-0.011306,0.035690,-0.005810,0.004506,-0.037956,0.001139
1,"This quilt, that my mother made, \n \n Still m...",Regency Romance: Model the text on a Regency r...,"The softest brown and brightest blue quilt, cr...",nbroad_1,-0.007521,-0.082697,0.041606,0.048020,0.007870,-0.018765,...,-0.046357,-0.019648,-0.044074,-0.049267,0.000438,0.047913,0.042219,0.012958,-0.048047,0.006332
2,It's the job of our agency to keep track of th...,Write like Ernest Hemingway: Focus on Hemingwa...,The agency's responsibility is to track and co...,nbroad_1,-0.012435,-0.057448,0.041290,0.021997,-0.005588,-0.011431,...,-0.009061,-0.024727,-0.005645,-0.049009,-0.007487,0.029255,0.023511,0.006769,-0.041317,-0.012102
3,"The first punch gets me right in the ribs, kno...",Grimm's Fairy Tales: Adapt the text to mimic t...,"In the sweltering sun, the stench of sweat and...",nbroad_1,0.023922,-0.051777,0.037086,0.057018,0.006423,0.006742,...,-0.024975,0.006210,-0.010690,-0.057320,0.015437,0.041056,0.012486,-0.006540,-0.030836,-0.000440
4,Some nights I lay awake staring at the ceiling...,High Fantasy Epic: Transform the essay into a ...,In the tapestry of the ethereal realm of Eldri...,nbroad_1,-0.002841,-0.056669,0.049056,0.072066,-0.006430,-0.020034,...,-0.031516,0.014793,-0.024928,-0.052689,-0.005655,0.044724,0.016817,-0.001000,-0.026472,0.008088


#Formatting the dataset
- Using only the needed columns.
- The dataset was provided with embeddings. I decided to tokenize and vectorize using the model provided in the huggingface platform. Therefore, embeddings are ignored.
- To avoid memory error while using Kaggle free tier or Colab free tier. The tokens were counted and selected no more than 600 or 700.

In [ ]:
df1= df[["dataset_id", "original_text", "rewrite_prompt","rewritten_text"]]


In [ ]:
df1['token_count'] = df1["original_text"].str.replace(',','').str.split().str.len()
df1['token_count_rewrit'] = df1["rewritten_text"].str.replace(',','').str.split().str.len()

<ipython-input-13-3313dc9a2b63>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['token_count'] = df1["original_text"].str.replace(',','').str.split().str.len()
<ipython-input-13-3313dc9a2b63>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['token_count_rewrit'] = df1["rewritten_text"].str.replace(',','').str.split().str.len()


In [ ]:
df1.shape

(58174, 6)

In [ ]:
df2= df1[(df1["token_count"]<600)&(df1["token_count_rewrit"]<600)]

In [ ]:
"""
The prompts for training will follow the reported template on the Gemini Paper. To leverage transfer learning and not changing the initial format.
"""

gemini_prompts= """
<start_of_turn>user
You are a reverse prompt engineer. Read the following "Original Text" and create the correct "Instruction" that instructs a LLM to generate the the "Rewriten Text" accurately.
\n Original Text:\n{original_text}
\n Rewriten Text:\n{rewritten_text}
<end_of_turn>
<start_of_turn>model
\nInstruction:\n{rewrite_prompt}
"""

In [ ]:
df2["prompt"]= df2.apply(lambda row: gemini_prompts.format(original_text=row.original_text,
                                                             rewritten_text=row.rewritten_text,
                                                             rewrite_prompt=row.rewrite_prompt), axis=1)


<ipython-input-17-e54883f677b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["prompt"]= df2.apply(lambda row: gemini_prompts.format(original_text=row.original_text,


In [ ]:
df2

,dataset_id,original_text,rewrite_prompt,rewritten_text,token_count,token_count_rewrit,prompt
0,host,"Dear Randy,\n\nI hope this letter finds you we...",Rephrase this letter to infuse it with an elfi...,"Dear Randy,\n\nMay this enchanted message find...",63,67,\n<start_of_turn>user\nYou are a reverse promp...
1,nbroad_1,"This quilt, that my mother made, \n \n Still m...",Regency Romance: Model the text on a Regency r...,"The softest brown and brightest blue quilt, cr...",153,203,\n<start_of_turn>user\nYou are a reverse promp...
3,nbroad_1,"The first punch gets me right in the ribs, kno...",Grimm's Fairy Tales: Adapt the text to mimic t...,"In the sweltering sun, the stench of sweat and...",284,258,\n<start_of_turn>user\nYou are a reverse promp...
5,nbroad_1,"I can hardly read the letter, because the hand...",Fairy Tale Villain: Use the menacing and craft...,"My hand quivered as I clutched the letter, the...",463,272,\n<start_of_turn>user\nYou are a reverse promp...
8,nbroad_1,`` Brigands and cutpurses have nothing on me. ...,"Drunkard: Infuse the essay with the rambling, ...","""Swerry brutes and cutthroat cutpurse, they ai...",260,172,\n<start_of_turn>user\nYou are a reverse promp...
...,...,...,...,...,...,...,...
58169,aatiffraz,I knew I only had 10 seconds to change things....,Rewrite this text in the style of a philosophi...,\n\n## The Knight's Tale of Temporal Flux:\n\n...,138,153,\n<start_of_turn>user\nYou are a reverse promp...
58170,aatiffraz,The rebels ran into the building lining up in ...,Restyle this text as if it were written by a p...,"\n\nSure, here is the text rewritten as if it ...",108,160,\n<start_of_turn>user\nYou are a reverse promp...
58171,aatiffraz,"Jeanne loomed over the coffin, a great nagging...",Restyle this text as if it were written by a p...,"\n\nSure, here is the text rewritten as if it ...",147,151,\n<start_of_turn>user\nYou are a reverse promp...
58172,aatiffraz,The officers examined the graphic scene. The v...,Translate the essence of this text into a pira...,"\n\n**Pirate Narrative:**\n\nAvast, me heartie...",102,118,\n<start_of_turn>user\nYou are a reverse promp...


In [ ]:
df2['token_count_promp'] = df2["prompt"].str.replace(',','').str.split().str.len()

<ipython-input-19-b5a124a41fb7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['token_count_promp'] = df2["prompt"].str.replace(',','').str.split().str.len()


In [ ]:
final_df = df2[df2["token_count_promp"]<600]

In [ ]:
sample_df = final_df.sample(3000)

In [ ]:
#Create a dataset from the pandas dataframe for easier training

from datasets import Dataset
dataset = Dataset.from_pandas(sample_df)

In [ ]:
dataset

Dataset({
    features: ['dataset_id', 'original_text', 'rewrite_prompt', 'rewritten_text', 'prompt', 'token_count', 'token_count_rewrit', 'token_count_promp', '__index_level_0__'],
    num_rows: 3000
})

In [ ]:
#Tokenizing the dataset

dataset = dataset.map(lambda samples: tokenizer(samples['prompt']), batched=True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['dataset_id', 'original_text', 'rewrite_prompt', 'rewritten_text', 'token_count', 'token_count_rewrit', 'prompt', 'token_count_promp', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 3000
})

In [ ]:
pip install trl
"""
Supervised fine-tuning (or SFT for short) is a crucial step in RLHF. In TRL we provide an easy-to-use API to create your SFT models and train them with few lines of code on your dataset.
SFT allows popular instruction or conversational formats:
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "..."}]}
these formats can be passed to SFTTrainer using :
tokenizer.apply_chat_template(chat, tokenize=False)
or a custom formating function:

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"### Question: {example['question'][i]}\n ### Answer: {example['answer'][i]}"
        output_texts.append(text)
    return output_texts

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
)

"""

In [ ]:
from trl import SFTTrainer

In [ ]:
import transformers
"""
- gradient_accumulation_steps allows for lower memory usage; instead of calculating the gradients for the whole batch at once to do it in smaller steps, when enough gradients are accumulated we run the model’s optimization step.
The reason to use gradient accumulation steps is when your whole batch size does not fit on one GPU, the training becomes slower
- SFT Trainer also accepts PEFT config to pass the LoRa configuration with: peft_config=peft_config
"""

trainer = SFTTrainer(
    model=model,
    dataset_text_field= "prompt",
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',


    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    train_dataset=dataset,
    peft_config=config,
    tokenizer=tokenizer,
    max_seq_length=600,

)


#The Regular HuggingFace trainer can also be used if wanted
"""trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
        max_length=600
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)"""
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast f

Step,Training Loss
1,1.865300
2,1.792400
3,1.772800
4,1.644000
5,1.944900
6,1.839800
7,1.821800
8,2.024300
9,1.534300
10,1.508600


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

TrainOutput(global_step=200, training_loss=1.5546031677722931, metrics={'train_runtime': 1363.5433, 'train_samples_per_second': 0.587, 'train_steps_per_second': 0.147, 'total_flos': 4392776392212480.0, 'train_loss': 1.5546031677722931, 'epoch': 0.27})

In [ ]:
dataset[4]["prompt"]

'\n<start_of_turn>user\nYou are a reverse prompt engineer. Read the following "Original Text" and create the correct "Instruction" that instructs a LLM to generate the the "Rewriten Text" accurately.\n\n Original Text:\nElizabeth Edwards, 49, and her 13-year-old daughter Katie were found dead in Spalding in April.\nThe 15-year-old denies murder, but has admitted manslaughter on the grounds of diminished responsibility, at Nottingham Crown Court.\nA boy, also 15, has already pleaded guilty to murder.\nNeither teenager can be named because of their ages.\nRead more about this and other stories from across Lincolnshire\nOn the second day of the trial a transcript of the girl\'s police interview was read.\nShe told officers she and her co-accused, who was 14 at the time, had planned the killing of Elizabeth and Katie Edwards in the days before their deaths on the evening of 13 April.\nShe also said they had attempted to carry out the killings on two other occasions.\nThe jury heard her tel

In [ ]:
gen_kwargs = {"min_length": -1, "top_p": 1.0, "pad_token_id": tokenizer.eos_token_id}

In [ ]:
"""
Some generation kwargs are use to improve the search in the inference step.
top_k:With `top_k` sampling, the output gets restricted the k most likely tokens
 do_sample: With sampling, the output is unexpected -- sometimes too unexpected.
For more information on generation strategies https://huggingface.co/docs/transformers/internal/generation_utils.
Added gen_kwargs to show both options on how they can be passed onto the generate function.

"""

batch = tokenizer(dataset[4]["prompt"].split("Instruction:")[0], return_tensors='pt')
batch.to("cuda")
with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch,  top_k=4, do_sample=True, max_new_tokens=50, **gen_kwargs)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 
user
You are a reverse prompt engineer. Read the following "Original Text" and create the correct "Instruction" that instructs a LLM to generate the the "Rewriten Text" accurately.

 Original Text:
Elizabeth Edwards, 49, and her 13-year-old daughter Katie were found dead in Spalding in April.
The 15-year-old denies murder, but has admitted manslaughter on the grounds of diminished responsibility, at Nottingham Crown Court.
A boy, also 15, has already pleaded guilty to murder.
Neither teenager can be named because of their ages.
Read more about this and other stories from across Lincolnshire
On the second day of the trial a transcript of the girl's police interview was read.
She told officers she and her co-accused, who was 14 at the time, had planned the killing of Elizabeth and Katie Edwards in the days before their deaths on the evening of 13 April.
She also said they had attempted to carry out the killings on two other occasions.
The jury heard her tell police her co-accused had

In [ ]:
result= tokenizer.decode(output_tokens[0], skip_special_tokens=True)

In [ ]:
import re
# Use regular expression to extract text between "\nInstruction:\n" and the next "\n"
instruction_text = re.search(r'\nInstruction:\n(.*?)\n', result, re.DOTALL)
if instruction_text:
    instruction_text = instruction_text.group(1).strip()

print(instruction_text)

Rewrite this as a visionary urban planner's plan for the future of the city.
